In [ ]:
# Imports

import socket
from emoji import demojize
import pandas as pd
import re
from datetime import datetime, timedelta

In [ ]:
# Constants, hide these

server = 'irc.chat.twitch.tv'
port = 6667
channel = 'sequisha'
nickname = ''
token = ''
data = []
delta = timedelta(minutes = 1)

In [ ]:
# Method to add comments to data frame
# format for regex reference :larocke!larocke@larocke.tmi.twitch.tv PRIVMSG #larocke :8

def get_chat_dataframe(line):
    now = datetime.now()
    try:

        username, channel, message = re.search(':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', line).groups()

        d = {
            'dt': now,
            'channel': channel,
            'username': username,
            'message': message
        }

        data.append(d)
            
    except Exception:
        pass
    
    # Once a comment is added anything older than time delta is deleted. Will only work for big stream.
    
    for i in data:
        if i['dt'] < now - delta:
            data.remove(i)
            
    
        

In [ ]:
# Imports Initializations

s = socket.socket()
s.connect((server, port))

In [ ]:
# Sending oatharization info to twitch

s.send(f"PASS {token}\n".encode('utf-8'))
s.send(f"NICK {nickname}\n".encode('utf-8'))
s.send(f"JOIN #{channel}\n".encode('utf-8'))

In [ ]:
resp = s.recv(2048).decode('utf-8')

In [ ]:
# Main loop that writes to chat.log

while True:
    resp = s.recv(2048).decode('utf-8')

    if resp.startswith('PING'):
        s.send("PONG\n".encode('utf-8'))
    
    elif len(resp) > 0:
        get_chat_dataframe(demojize(resp))